## Method Resolution Order

How does Python know the order to call the methods?  In our example, it wasn't critical, but sometimes it might be critical to know the order of resolving calls to `super`.  That is where the Python *method resolution order* comes in.  A property of a class is its *method resolution order*, accessible in the `__mro__` attribute.

First we need to define all of our classes as in the previous lecture:

In [ ]:
import numpy as np
from matplotlib.pyplot import figure, legend, plot
%matplotlib inline

class Forest(object):
    """ A Forest can grow but won't burn.
    """

    def __init__(self, size=(150, 150), p_sapling=0.0025):
        self.size = size
        self.trees = np.zeros(self.size, dtype=bool)
        self.p_sapling = p_sapling

    @property
    def name(self):
        return self.__class__.__name__

    @property
    def num_cells(self):
        return self.size[0] * self.size[1]

    @property
    def tree_fraction(self):
        return self.trees.sum() / float(self.num_cells)

    def grow_trees(self):
        growth_sites = self._rand_bool(self.p_sapling)
        self.trees[growth_sites] = True

    def _rand_bool(self, p):
        return np.random.uniform(size=self.trees.shape) < p

    def advance_one_step(self):
        self.grow_trees()

class MoldProneForest(Forest):
    """Tree-killing mold can grow in tree groves of a certain size.
    """
    def __init__(self, p_mold=3.0e-3, critical_density=3, *args, **kwargs):
        super(MoldProneForest, self).__init__(*args, **kwargs)
        self.p_mold = p_mold
        self.critical_density = critical_density
        self.mold = np.zeros(self.size, dtype=bool)

    @property
    def mold_fraction(self):
        return self.mold.sum() / float(self.num_cells)

    @property
    def losses(self):
        return self.mold

    def advance_one_step(self):
        self.grow_trees()
        self._grow_mold_and_kill_trees()

    def _grow_mold_and_kill_trees(self):
        working_size = (self.size[0] + 2, self.size[1] + 2)
        tmp_trees = np.zeros(working_size, dtype=np.int8)
        tmp_trees[1:-1, 1:-1] = self.trees

        north = tmp_trees[:-2, 1:-1]
        south = tmp_trees[2:, 1:-1]
        east = tmp_trees[1:-1, :-2]
        west = tmp_trees[1:-1, 2:]
        density = north + south + east + west
        mold_prone_trees = density >= self.critical_density

        self.mold = self._rand_bool(self.p_mold) & mold_prone_trees
        self.trees[self.mold] = False
        
class BurnableForest(Forest):
    """ BurnableForest supports fires.
    """

    def __init__(self, p_lightning=5.0e-6, *args, **kwargs):
        super(BurnableForest, self).__init__(*args, **kwargs)
        self.p_lightning = p_lightning
        self.fires = np.zeros(self.size, dtype=bool)

    @property
    def fire_fraction(self):
        return self.fires.sum() / float(self.num_cells)

    def start_fires(self):
        lightning_strikes = self._rand_bool(self.p_lightning) & self.trees
        self.fires[lightning_strikes] = True

    def burn_trees(self):
        """The rules for burning trees:
        -New fires start in tree-filled adjacent cells.
        -Trees in currently burning cells burn and disappear.

        The fire-spreading algorithm is calculated on an array padded
        around the edges with zeros to allow vectorized computations.
        """
        working_size = (self.size[0] + 2, self.size[1] + 2)
        fires = np.zeros(working_size, dtype=bool)
        fires[1:-1, 1:-1] = self.fires
        north = fires[:-2, 1:-1]
        south = fires[2:, 1:-1]
        east = fires[1:-1, :-2]
        west = fires[1:-1, 2:]
        new_fires = (north | south | east | west) & self.trees
        self.trees[self.fires] = False
        self.fires = new_fires

    def advance_one_step(self):
        self.grow_trees()
        self.start_fires()
        self.burn_trees()

    @property
    def losses(self):
        return self.fires

class InstantBurnForest(BurnableForest):
    """BurnableForest where fires spread instantaneously relative to the
    growth of new saplings.
    """
    def advance_one_step(self):
        self.grow_trees()
        self._strike_and_burn()

    def _strike_and_burn(self):
        strikes = self._rand_bool(self.p_lightning) & self.trees
        groves, num_groves = label(self.trees)
        fires = set(groves[strikes])
        self.fires.fill(False)
        for fire in fires:
            self.fires[groves == fire] = True
        self.trees[self.fires] = False

class MoldyInstantBurningForest(InstantBurnForest, MoldProneForest):
    @property
    def losses(self):
        return self.fires | self.mold

    def advance_one_step(self):
        self.grow_trees()
        self._grow_mold_and_kill_trees()
        self._strike_and_burn()

Now we can look at the `__mro__` attribute of the `MoldyInstantBurningForest`, which has complex ancestry:

In [ ]:
MoldyInstantBurningForest.__mro__

This explains the call to `super`.  The first argument of `super` is the current location in the method resolution order.  The method resolution order is found in the `__class__` attribute of `self` class whose method resolution order you want to use.

Let's think about what happens when you create a new instance of MoldyInstantBurningForest:

In [ ]:
MoldyInstantBurningForest.__init__??

Check the class definitions for the classes listed in the method resolution order.  Neither `MoldyInstantBurningForest` nor `InstantBurnForest` defines an `__init__` method, so what we see is the `__init__` method of `BurnableForest`.  Inspection of the constructor code shows a call to `super`.  Now let's step through the code to trace our way through the constructor calls to see what happens with `super`.

In [ ]:
import pdb
pdb.runcall(MoldyInstantBurningForest)

`super` in `BurnableForest` looks at the `__mro__` of `self`, which is a `MoldyInstantBurningForest` instance.  The next item after `BurnableForest` is `MoldProneForest`, so that is the `__init__` that is called.

The call from `MoldProneForest` will take the next one after `MoldProneForest`, which is `Forest`.

So that first argument in `super` is kind of like a locator.  It tells the function where it is in the method resolution order of `self`.

As long as there is a chain of calls to `super()`, all of the methods of a class hierarchy can be called.

Note the effect of order in the inheritance specification in the class definition.  If we reversed the order and put `MoldProneForest` first, then it would have been the first constructor to be called.

Note also that to get this behavior, it is necessary that the top-most parent class inherit from `object`.

Copyright 2008-2016, Enthought, Inc.<br>Use only permitted under license.  Copying, sharing, redistributing or other unauthorized use strictly prohibited.<br>http://www.enthought.com